### 📦載入套件

In [4]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

### 📃抓取 PTT「HatePolitics」板單個頁面文章資訊的函式
#####  步驟分別是：
- 一個頁面會有好幾篇文章，需要點擊文章網址後，才能抓取文章內容
- 點擊網址後，簡單驗證文章以及抓取文章內文
- 最後以清單形式返回所有文章的資訊

In [5]:
#   建立函式
def get_ptt_post(soup):

    #   取得文章列表
    #   目標 div
    data = soup.select("div.r-ent")
    result = []
    for i in data:

        #   網址
        #   PTT 網站
        # 檢查選擇器是否找到任何元素
        title_a_elements = i.select("div.title a")
        if title_a_elements:  # 如果列表不為空
            oriLink = "https://www.ptt.cc" + title_a_elements[0]["href"]
        else:
            # 處理找不到元素的情況，例如跳過此文章
            print("跳過此文章：找不到 'div.title a' 元素")
            continue  # 移至下一篇文章
        #   取得文章相關資訊
        #   內文
        #   請求文章內文
        res_content = requests.get(oriLink)
        soup_content = bs(res_content.text,"lxml")
        #   文章內容簡易驗證
        results_content = soup_content.select('span.article-meta-value')
        if len(results_content) > 3:
            #   驗證成功, 篩出文章內文
            content = soup_content.find(id="main-content").text
            Footer = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
            #   移除註腳以下內容
            content = content.split(Footer)
            #   存取內容
            main_content = content[0]
            pass
        else:
            #   驗證失敗, 跳過該文章
            print(oriLink,"內文異常:ID/版標/標題/日期為空")
            print(results_content)
            continue

        result.append({
            "內文":main_content
        })
    return result

### 📃抓取 PTT「HatePolitics」板的多頁文章資訊
#####  工作流程：
- 前處理、解析網址
- 自動抓取首頁前 N 頁的文章（N為手動輸入的頁數）
- 將多頁文章整合成一個表格（DataFrame）

In [6]:
#   前綴與首頁網址
prefix = "https://www.ptt.cc"
url = "https://www.ptt.cc/bbs/HatePolitics/index.html"

#   請求與解析
res = requests.get(url)
soup = bs(res.text,"lxml")


#   建立資料集並匯出
#   首次呼叫 Def
output = []
result = get_ptt_post(soup)

#   將結果函式返回的內容存入output陣列
output += result

#   再次呼叫數(N)頁 - 前頁網址
N = 30
previous_page = soup.select("div#action-bar-container div.action-bar div.btn-group-paging a")[1]["href"]
# Extract the page number using split and indexing
page_number = int(previous_page.split('/')[-1].split('.')[0].replace('index',''))

for i in range(1,N):
    url = "https://www.ptt.cc/bbs/HatePolitics/index{}.html".format(page_number-i)
    res = requests.get(url)
    soup = bs(res.text,"lxml")

    result = get_ptt_post(soup)
    output += result
    print("{} is ok".format(url))

#   產出資料集
df = pd.DataFrame(output)

https://www.ptt.cc/bbs/HatePolitics/index4121.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4120.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4119.html is ok
跳過此文章：找不到 'div.title a' 元素
https://www.ptt.cc/bbs/HatePolitics/index4118.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4117.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4116.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4115.html is ok
跳過此文章：找不到 'div.title a' 元素
https://www.ptt.cc/bbs/HatePolitics/index4114.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4113.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4112.html is ok
跳過此文章：找不到 'div.title a' 元素
https://www.ptt.cc/bbs/HatePolitics/index4111.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4110.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4109.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4108.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4107.html is ok
https://www.ptt.cc/bbs/HatePolitics/index4106.html is ok
https:/

 ### 🧻清理並格式化文章內文
 #####  使用正則表達處理內文的內容：
 - 先確認內文欄位存在
 - 一共使用三種方式整理內文：
   1. 不屬於內文的文字都刪除
   2. 內文只保留中文字以及數字
   3. 把文章是新聞的都刪除
 - 將結果存在"clean_processed_ptt_tech_job_post.csv"

In [7]:

if '內文' in df.columns:
    def process_text(text):
        # 檢查是否為string
        if isinstance(text, str):
            #用正則表達處理內文
            pattern = r'標題(.*?)時間.*?\n(.*)'
            match = re.search(pattern, text, re.DOTALL)
            if match:
                title = match.group(1).strip()
                content = match.group(2).strip()
                return f'{title}\n{content}'
        return text  # 如果不匹配，返回原文本或者非字符串類型

    # 用re處理每行
    df['內文'] = df['內文'].apply(process_text)


In [8]:
clean_text = []
for text in df['內文']:
  if isinstance(text, str):
    clean = re.sub(r'(\n|\t|\r|[a-z]|[A-Z]|http\S+)', r'', text)
    clean_text.append(clean)
  else:
    clean_text.append('')  

# 清理完的text存成新的一欄 data
df['內文'] = clean_text

In [9]:
output_file = './src/clean_processed_ptt_tech_job_post.csv'
# 檢查 '內文' 欄位是否存在
if '內文' in df.columns:
    # 篩選資料，移除含有 "[新聞]" 的列
    filtered_df = df[~df['內文'].str.contains(r'\[新聞\]', na=False)]
    # 儲存到新的CSV檔案
    filtered_df.to_csv(output_file, index=False)

print(f"已清除內文所有雜物，結果儲存至 {output_file}")

已清除內文所有雜物，結果儲存至 ./src/clean_processed_ptt_tech_job_post.csv
